In [1]:
import warnings
warnings.filterwarnings('ignore')

import nltk
# nltk.download('punkt')          #After you have downloaded this please commnent
# nltk.download('wordnet')        #After you have downloaded this please commnent
# nltk.download('omw-1.4')          #After you have downloaded this please commnent
from nltk.stem import WordNetLemmatizer

import json
import pickle

import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import SGD

# **CHATBOT - TRAIN IT WITH YOUR OWN QUESTIONS**

## **¿What do you need to train a chatbot?**


<font size="3"> There are a lot of ways to create chatbots but most of the ways require too much data and computing. In this notebook, we’ll show a simplistic chatbot using an intent-based database. You can modify the dataset and try the chatbot on your own problems. The dataset is contained in the intents.json file. 

In this chatbot, we are not trying to generate the text by understanding the question but we are only trying to understand the intent of the question or trying to predict the tag of the question. Knowing the tag can give us the idea of what kind of question are we talking about, once the tags are identified, a response can be picked up from the sample responses attached with each tag. So, in the true sense, it is a tag predictor problem but if the dataset is formatted properly it can work as a brilliant chatbot, it may not work as a general-purpose chatbot but it can definitely work as a task-based chatbot meaning a chatbot trained to answer questions on one or two topics. Real chatbots which function like Siri or OK Google require terabytes of training data thus creating a chatbot with intent is the best option for people with less computing power.<br><br>
    
Without further ado let’s dive into our code. We already Loaded the libraries. Let's load the dataset, which is in the format of a JSON file, and see what our data looks like. <br><br>
    
Credits: https://medium.com/aiguys/an-intent-based-chatbot-7efd08faa47c

In [2]:
data_file = open('intents.json').read()
intents = json.loads(data_file)
intents['intents']

[{'tag': 'greeting',
  'patterns': ['Hi there',
   'How are you',
   'Is anyone there?',
   'Hey',
   'Hola',
   'Hello',
   'Good day'],
  'responses': ['Hi there, how can I help?'],
  'context': ['']},
 {'tag': 'goodbye',
  'patterns': ['Bye',
   'See you later',
   'Goodbye',
   'Nice chatting to you, bye',
   'Till next time'],
  'responses': ['Bye! Have a nice day'],
  'context': ['']},
 {'tag': 'thanks',
  'patterns': ['Thanks',
   'Thank you',
   "That's helpful",
   'Awesome, thanks',
   'Thanks for helping me'],
  'responses': ['Happy to help!'],
  'context': ['']},
 {'tag': 'noanswer',
  'patterns': [],
  'responses': ["Sorry, can't understand you"],
  'context': ['']}]

<font size="3"> You can see that the structure of the dataset is built on 4 elements:<br><br>
**1) Tag**: The different categories you would like to have. Some might be greeting, thanks, etc.<br><br>
**2) Patterns**: Different texts or sentences that belong to a specific tag. <br><br>
**3) Responses**: The answer or answers for the questions or sentences in a specific tag. <br><br>
**4) Context**: The conversation context. We are leaving it empty in this example. <br><br>
    
Let's see these four elements for the first tag in our dataset:

In [3]:
intents['intents'][0]

{'tag': 'greeting',
 'patterns': ['Hi there',
  'How are you',
  'Is anyone there?',
  'Hey',
  'Hola',
  'Hello',
  'Good day'],
 'responses': ['Hi there, how can I help?'],
 'context': ['']}

<font size="3"> Here in the greeting tag we have 7 different questions. They are all written under the patterns element. For all those question there will be a single answer. In this case it will be: Hi there, how can I help?

In [4]:
intents['intents'][0]['patterns']

['Hi there',
 'How are you',
 'Is anyone there?',
 'Hey',
 'Hola',
 'Hello',
 'Good day']

In [5]:
intents['intents'][0]['responses']

['Hi there, how can I help?']

## **Preprocessing**

<font size="3"> Now that we are familiar with the data structure, let's dig a little bit on text pre-processing. In this project we will specifically use two tools:<br><br>
    
**1) Tokenization**<br><br>
**2) Lemmatization**<br><br>
    
### **Word_tokenize**

<font size="3"> In Natural Language Processing, tokenization divides a string into a list of tokens. Tokens come in handy when finding valuable patterns and helping to replace sensitive data components with non-sensitive ones. Tokens can be though of as a word in a sentence or a sentence in a paragraph. Let's see an example:   

In [6]:
example_tokenize = nltk.word_tokenize("Is anyone there?")
print(example_tokenize)

['Is', 'anyone', 'there', '?']


In [7]:
#Create an empty list and add de different tokens
words = []
words.extend(example_tokenize)
print(words)

['Is', 'anyone', 'there', '?']


## **Tokenize all information**

<font size="3"> Let's work on our dataset and tokenize the different texts, sentences or questions that we have.<br><br>

In [8]:
words = []
classes = []
documents = []
ignore_words = ['?', '!']

# intents: Groups of conversations. Recall that each intent has 4 elements: 1) Tag, 2) Patterns, 3) Responses, 4) Context
# patterns: Different texts, sentences or questions related to a tag.

for intent in intents['intents']:
    for pattern in intent['patterns']:

        # Tokenize each text, sentence or questions
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Append all tokens to a document
        documents.append((w, intent['tag']))

        # Adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [9]:
print(words)

['Hi', 'there', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hey', 'Hola', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Nice', 'chatting', 'to', 'you', ',', 'bye', 'Till', 'next', 'time', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Awesome', ',', 'thanks', 'Thanks', 'for', 'helping', 'me']


In [10]:
documents

[(['Hi', 'there'], 'greeting'),
 (['How', 'are', 'you'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hey'], 'greeting'),
 (['Hola'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'later'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'),
 (['Till', 'next', 'time'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Awesome', ',', 'thanks'], 'thanks'),
 (['Thanks', 'for', 'helping', 'me'], 'thanks')]

In [11]:
classes

['greeting', 'goodbye', 'thanks']

## **Lemmatization**

<font size="3"> Lemmatization is the process of grouping together the different inflected forms of a word so they can be analyzed as a single item. Putting it differently: Lemmatization is the process of converting a word to its base form. Lemmatization is similar to stemming but it brings context to the words. So it links words with similar meanings to one word. The difference between stemming and lemmatization is, lemmatization considers the context and converts the word to its meaningful base form, whereas stemming just removes the last few characters, often leading to incorrect meanings and spelling errors.

For example, lemmatization would correctly identify the base form of ‘caring’ to ‘care’, whereas, stemming would cutoff the ‘ing’ part and convert it to car. ‘Caring’ -> Lemmatization -> ‘Care’ ‘Caring’ -> Stemming -> ‘Car’

In [12]:
lemmatizer = WordNetLemmatizer()

print("rocks :", lemmatizer.lemmatize("rocks"))
print("corpora :", lemmatizer.lemmatize("corpora"))
print("bats :", lemmatizer.lemmatize("Bats".lower()))
print("feet :",lemmatizer.lemmatize("feet"))

rocks : rock
corpora : corpus
bats : bat
feet : foot


<font size="3"> This kind of problems can be corrected if we provide the correct ‘part-of-speech’ tag (POS tag) as the second argument to lemmatize().

Sometimes, the same word can have a multiple lemmas based on the meaning / context. <br><br>
    
Credits: https://www.machinelearningplus.com/nlp/lemmatization-examples-python/

In [13]:
print("stripes :", lemmatizer.lemmatize("stripes","v"))
print("stripes :", lemmatizer.lemmatize("stripes","n"))

stripes : strip
stripes : stripe


In [14]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
print(words)

['hi', 'there', 'how', 'are', 'you', 'is', 'anyone', 'there', 'hey', 'hola', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'goodbye', 'nice', 'chatting', 'to', 'you', ',', 'bye', 'till', 'next', 'time', 'thanks', 'thank', 'you', 'that', "'s", 'helpful', 'awesome', ',', 'thanks', 'thanks', 'for', 'helping', 'me']


## **Pickle**

<font size="3"> Pickle is used for serializing and de-serializing Python object structures, also called marshalling or flattening. Serialization refers to the process of converting an object in memory to a byte stream that can be stored on disk or sent over a network. Later on, this character stream can then be retrieved and de-serialized back to a Python object. Pickling is not to be confused with compression! The former is the conversion of an object from one representation (data in Random Access Memory (RAM)) to another (text on disk), while the latter is the process of encoding data with fewer bits, in order to save disk space. Pickling is useful for applications where you need some degree of persistency in your data. Your program's state data can be saved to disk, so you can continue working on it later on. It can also be used to send data over a Transmission Control Protocol (TCP) or socket connection, or to store python objects in a database. Pickle is very useful for when you're working with machine learning algorithms, where you want to save them to be able to make new predictions at a later time, without having to rewrite everything or train the model all over again. <br><br>
    
<font size="3">To open the file for writing, simply use the open() function. The first argument should be the name of your file. The second argument is 'wb'. The w means that you'll be writing to the file, and b refers to binary mode. This means that the data will be written in the form of byte objects. If you forget the b, a TypeError: must be str, not bytes will be returned. You may sometimes come across a slightly different notation; w+b, but don't worry, it provides the same functionality. Once the file is opened for writing, you can use pickle.dump(), which takes two arguments: the object you want to pickle and the file to which the object has to be saved. In this case, the former will be dogs_dict, while the latter will be outfile.

In [15]:
pickle.dump(words, open('words.pkl','wb'))
pickle.dump(classes, open('classes.pkl','wb'))

## **Training data**

<font size="3"> Now we will get the information from the intents.json file and put it in an appropiate format to train a deep neural network. What the following code does can be summarized as follows:<br><br>

<font size="3">
1) Look in the list of all words an see if each word in this list appears in the patterns we have.<br>
2) If they appear store this with 1 in an array. Otherwise put 0.<br>
3) Continue with this procedure until the end of the words list. You will get an array of 1s and 0s. <br>
4) Create a new array that has as many dimensions as classes in the classes list. In this example there are 3 classes. <br>
5) If you are in the first class, this array will be [1,0,0]. if you are in the second class it will be [0,1,0] and so on. <br>
    

In [16]:
#Training data preparation

training = []
output_empty = [0] * len(classes)

for doc in documents:
    # bag of words
    bag = []
    # patterns
    pattern_words = doc[0]
    # lemmatization of each token in patterns
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    #If a word in the list words is in pattern_words put 1, otherwise put 0
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    output_row = list(output_empty)
    #Put number 1 in the position of the vector that corresponds to the class we are iterating
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [17]:
#Define training as an array
training = np.array(training)

In [18]:
#The first element of training contains 2 list:
#First list: Array with 0 or 1 if each word on words was in the first pattern or not
#Second list: Array with 0 or 1 that represents the class. Here we are in the first class of a total of 3 classes
training[0]

array([list([1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
       list([1, 0, 0])], dtype=object)

In [19]:
# train_x: patterns
# train_y: class
train_x = list(training[:,0])
train_y = list(training[:,1])

In [20]:
print('words: '+str(words))
print('--------------------------------------------------------------------------------------------')
print('pattern: '+str(documents[0]))
print('train_x: '+str(train_x[0]))
print('train_y: '+str(train_y[0]))

#In this example the list word contains 3 words that are in the first pattern: "hi", "there", "there"
#Therefore the arry has three "1s" and the rest are "0s"
#Since that corresponds to the first tag, train_y has 1 in the first element and the others are 0

words: ['hi', 'there', 'how', 'are', 'you', 'is', 'anyone', 'there', 'hey', 'hola', 'hello', 'good', 'day', 'bye', 'see', 'you', 'later', 'goodbye', 'nice', 'chatting', 'to', 'you', ',', 'bye', 'till', 'next', 'time', 'thanks', 'thank', 'you', 'that', "'s", 'helpful', 'awesome', ',', 'thanks', 'thanks', 'for', 'helping', 'me']
--------------------------------------------------------------------------------------------
pattern: (['Hi', 'there'], 'greeting')
train_x: [1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
train_y: [1, 0, 0]


## **Deep neural network**

<font size="3"> Now we will build a very simple architechture. We will have a DNN which contains:<br><br>

a) A hidden layer with 128 nodes. The input shape in this case is 40, which corresponds to the number of elements in the first array from train_x. Activation functions is relu.<br>
b) A hidden layer with 64 nodes.<br>
c) An output layer with a shape equal to the shape of train_y. In this case 3 elements.It will predict to which class the words belong. 
    
You can look at a simple examplanation with an example in the following link:
    
https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

In [21]:
# Deep Neural Network
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5248      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 3)                 195       
                                                                 
Total params: 13,699
Trainable params: 13,699
Non-trainable params: 0
_________________________________________________________________


In [22]:
#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=300, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/300
4/4 [==============================] - 1s 6ms/step - loss: 1.1045 - accuracy: 0.2941
Epoch 2/300
4/4 [==============================] - 0s 5ms/step - loss: 1.0668 - accuracy: 0.5294
Epoch 3/300
4/4 [==============================] - 0s 6ms/step - loss: 1.0288 - accuracy: 0.4706
Epoch 4/300
4/4 [==============================] - 0s 4ms/step - loss: 0.9568 - accuracy: 0.6471
Epoch 5/300
4/4 [==============================] - 0s 5ms/step - loss: 0.7777 - accuracy: 0.6471
Epoch 6/300
4/4 [==============================] - 0s 4ms/step - loss: 0.8435 - accuracy: 0.7059
Epoch 7/300
4/4 [==============================] - 0s 3ms/step - loss: 0.7508 - accuracy: 0.7647
Epoch 8/300
4/4 [==============================] - 0s 5ms/step - loss: 0.8215 - accuracy: 0.7059
Epoch 9/300
4/4 [==============================] - 0s 5ms/step - loss: 0.8097 - accuracy: 0.5882
Epoch 10/300
4/4 [==============================] - 0s 7ms/step - loss: 0.7574 - accuracy: 0.7059
Epoch 11/300
4/4 [===========

4/4 [==============================] - 0s 4ms/step - loss: 0.0543 - accuracy: 1.0000
Epoch 85/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0252 - accuracy: 1.0000
Epoch 86/300
4/4 [==============================] - 0s 3ms/step - loss: 0.0080 - accuracy: 1.0000
Epoch 87/300
4/4 [==============================] - 0s 5ms/step - loss: 0.0083 - accuracy: 1.0000
Epoch 88/300
4/4 [==============================] - 0s 7ms/step - loss: 0.0129 - accuracy: 1.0000
Epoch 89/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0277 - accuracy: 1.0000
Epoch 90/300
4/4 [==============================] - 0s 2ms/step - loss: 0.0189 - accuracy: 1.0000
Epoch 91/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0122 - accuracy: 1.0000
Epoch 92/300
4/4 [==============================] - 0s 5ms/step - loss: 0.0212 - accuracy: 1.0000
Epoch 93/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 94/300
4/4 [===============

4/4 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 168/300
4/4 [==============================] - 0s 5ms/step - loss: 7.1024e-04 - accuracy: 1.0000
Epoch 169/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 170/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0042 - accuracy: 1.0000
Epoch 171/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0326 - accuracy: 1.0000
Epoch 172/300
4/4 [==============================] - 0s 3ms/step - loss: 0.0194 - accuracy: 1.0000
Epoch 173/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 174/300
4/4 [==============================] - 0s 5ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 175/300
4/4 [==============================] - 0s 2ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 176/300
4/4 [==============================] - 0s 3ms/step - loss: 0.0300 - accuracy: 1.0000
Epoch 177/300
4/4 [=

4/4 [==============================] - 0s 3ms/step - loss: 0.0068 - accuracy: 1.0000
Epoch 250/300
4/4 [==============================] - 0s 4ms/step - loss: 8.3278e-04 - accuracy: 1.0000
Epoch 251/300
4/4 [==============================] - 0s 4ms/step - loss: 3.3413e-04 - accuracy: 1.0000
Epoch 252/300
4/4 [==============================] - 0s 3ms/step - loss: 0.0023 - accuracy: 1.0000
Epoch 253/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0238 - accuracy: 1.0000
Epoch 254/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0019 - accuracy: 1.0000
Epoch 255/300
4/4 [==============================] - 0s 3ms/step - loss: 0.0156 - accuracy: 1.0000
Epoch 256/300
4/4 [==============================] - 0s 3ms/step - loss: 3.4397e-04 - accuracy: 1.0000
Epoch 257/300
4/4 [==============================] - 0s 3ms/step - loss: 0.0020 - accuracy: 1.0000
Epoch 258/300
4/4 [==============================] - 0s 4ms/step - loss: 0.0045 - accuracy: 1.0000
Epoch 259/30